# Data preprocessing

Data preprocessing for MUDI data and Human Connectome Project (HCP)

In [ ]:
import os
import subprocess
from dataclasses import dataclass
from pathlib import Path
from typing import Optional

import h5py
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import torch
from nilearn import image, plotting
from nilearn.masking import apply_mask
from plotly.subplots import make_subplots

from autoencoder.logger import logger, set_log_level

**Change these to your correct path**

Make sure you have the following packages installed:
 - FSL: https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/

In [ ]:
MUDI_PATH = Path("/media/maarten/disk1/MUDI/")
HCP_PATH = Path("/media/maarten/disk1/HCP/")

MUDI_OUTPUT_PATH = Path("/media/maarten/disk1/data_mudi.hdf5")
HCP_OUTPUT_PATH = Path("/media/maarten/disk1/data_hcp.hdf5")

FSL_INSTALL_PATH = "/usr/local/fsl"

In [ ]:
os.environ["FSLOUTPUTTYPE"] = "NIFTI_GZ"
os.environ["FSLDIR"] = FSL_INSTALL_PATH
os.environ["PATH"] = f"{FSL_INSTALL_PATH}/bin:" + os.environ["PATH"]

In [ ]:
set_log_level(10)

## Step 1: Retrieving file paths

We create a `dataclass` to store all the paths in. We can use these later to load the files.

In [ ]:
@dataclass
class MRISubjectData:
    subject_name: str
    subject_id: int
    project: str  # Name of the dataset
    output_file: Path

    # paths
    scheme_path: Path
    root_path: Path  # path were all MRI data of the subject is located
    dmri_path: Path  # relative to root_path
    t1_path: Path  # relative to root_path
    t2_path: Optional[Path] = None  # relative to root_path
    brain_mask_path: Optional[Path] = None  # relative to root_path
    fivett_mask_path: Optional[Path] = None

    # normalization data
    max_data: Optional[float] = 1.0
    lstq_coefficient: Optional[float] = 1.0

In [ ]:
mri_mudi_subjects: list[MRISubjectData] = list()
mri_hcp_subjects: list[MRISubjectData] = list()

# MUDI subjects
for i in range(1, 6):
    subject = f"cdmri001{i}"

    # check if we have a previously generated 5tt mask
    fivett_mask_path = None
    if Path(MUDI_PATH, subject, "5tt.nii").exists():
        fivett_mask_path = Path(MUDI_PATH, subject, "5tt.nii")

    mri_mudi_subjects.append(
        MRISubjectData(
            subject,
            10 + i,
            "MUDI",
            MUDI_OUTPUT_PATH,
            Path(MUDI_PATH, "parameters_new.txt"),
            Path(MUDI_PATH, subject),
            Path("MB_Re_t_moco_registered_applytopup.nii.gz"),
            Path("additional/anatomical.nii.gz"),
            brain_mask_path=Path("brain_mask.nii.gz"),
            fivett_mask_path=fivett_mask_path,
        )
    )

logger.info(f"found {len(mri_mudi_subjects)} MUDI subjects")

# HCP subjects
for i in range(1, 36):
    if i == 20:
        continue  # skip subject 20, it has fewer volumes than the rest

    subject = f"mgh_10{i:02d}"
    # search for the mri files
    dmri_path = list(HCP_PATH.glob(f"**/*{subject}_MR_ep2d_Diffusion_Gradwarped,_Eddy_Current_Corrected_Br_*.nii"))[0]
    t1_path = list(HCP_PATH.glob(f"**/*{subject}_MR_MPRAGE*.nii"))[0]

    # check if we have a previously generated brain mask
    brain_mask_path = None
    if Path(HCP_PATH, subject, "brain_mask.nii").exists():
        brain_mask_path = Path(HCP_PATH, subject, "brain_mask.nii")

    # check if we have a previously generated 5tt mask
    fivett_mask_path = None
    if Path(HCP_PATH, subject, "5tt.nii").exists():
        fivett_mask_path = Path(HCP_PATH, subject, "5tt.nii")

    # load and combine all MRI parameters
    scheme_path = Path(HCP_PATH, subject, f"{subject}_scheme.txt")
    if not scheme_path.exists():
        bvals_path = Path(HCP_PATH, subject, f"{subject}_bvals_ep2d_eddy.txt")
        bvecs_path = Path(HCP_PATH, subject, f"{subject}_bvecs_moco_norm_ep2d_eddy.txt")
        bvals = np.loadtxt(bvals_path)[:, np.newaxis]
        bvecs = np.loadtxt(bvecs_path)
        scheme = np.c_[bvecs, bvals]
        np.savetxt(scheme_path, scheme, fmt="%.6f %.6f %.6f %d")

    mri_hcp_subjects.append(
        MRISubjectData(
            subject,
            i,
            "HCP",
            HCP_OUTPUT_PATH,
            scheme_path,
            Path(HCP_PATH, subject),
            dmri_path.relative_to(Path(HCP_PATH, subject)),
            t1_path.relative_to(Path(HCP_PATH, subject)),
            brain_mask_path=brain_mask_path,
            fivett_mask_path=fivett_mask_path,
        )
    )

logger.info(f"found {len(mri_hcp_subjects)} HCP subjects")

## Step 2: Creating and applying a brain mask
Create a brain mask from dwi image if none was provided. Based on the script bet from FSL toolbox. Docs: https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/BET/UserGuide

In [ ]:
def generate_brain_mask(subject: MRISubjectData):
    """"""
    logger.info("starting brain mask generation")
    result = subprocess.run(
        [
            "bet",
            Path(subject.root_path, subject.dmri_path),
            Path(subject.root_path, "brain.nii"),
            "-m",
            "-n",
            "-f",
            "0.5",
        ]
    )
    if result.returncode == 0:
        subject.brain_mask_path = Path(subject.root_path, "brain_mask.nii.gz")
    logger.info("finished brain mask generation, return code: %d", result.returncode)
    return result.returncode

Apply the brain mask.

In [ ]:
def load_scan(subject: MRISubjectData) -> np.ndarray:
    dmri_path = Path(subject.root_path, subject.dmri_path)
    brain_mask_path = None
    if subject.brain_mask_path:
        brain_mask_path = Path(subject.root_path, subject.brain_mask_path)
    else:
        result_code = generate_brain_mask(subject)
        if result_code != 0:
            logger.error("could not generate brain mask")
            return
        brain_mask_path = Path(subject.root_path, subject.brain_mask_path)
    return np.transpose(apply_mask(imgs=str(dmri_path), mask_img=str(brain_mask_path)))

In [ ]:
def store_dmri_data(scan_data: np.ndarray, subject: MRISubjectData):
    logger.info("storing data for subject %s", subject.subject_name)

    index = np.full(shape=scan_data.shape[0], fill_value=subject.subject_id, dtype=np.int8)
    unnormalize_data = np.full(
        shape=(scan_data.shape[0], 2),
        fill_value=(subject.lstsq_coefficient, subject.max_data),
        dtype=np.float64,
    )

    with h5py.File(subject.output_file, "a") as file:
        if "data" not in file.keys():
            file.create_dataset("data", data=scan_data, chunks=True, maxshape=(None, scan_data.shape[1]))
            file.create_dataset("index", data=index, chunks=True, maxshape=(None,))
            file.create_dataset(
                "unnormalize_data", data=unnormalize_data, chunks=True, maxshape=(None, unnormalize_data.shape[1])
            )
        else:
            file["data"].resize(file["data"].shape[0] + scan_data.shape[0], axis=0)
            file["data"][-scan_data.shape[0] :] = scan_data

            file["index"].resize(file["index"].shape[0] + index.shape[0], axis=0)
            file["index"][-index.shape[0] :] = index

            file["unnormalize_data"].resize(file["unnormalize_data"].shape[0] + unnormalize_data.shape[0], axis=0)
            file["unnormalize_data"][-unnormalize_data.shape[0] :] = unnormalize_data


def normalize_data(subjects: list[MRISubjectData]):
    scheme = np.loadtxt(subjects[0].scheme_path)
    mask_b_0 = scheme[:, 3] == 0.0
    subject_0 = subjects[0]

    logger.info("normalizing subject %s (subject 0)", subject_0.subject_name)

    scan_data_subject_0 = load_scan(subject_0)

    max_data = np.percentile(scan_data_subject_0, 95)
    logger.info("95th percentile: %f", max_data)
    subject_0.max_data = max_data
    subject_0.lstsq_coefficient = 1.0

    scan_data_median_subject_0 = np.median(scan_data_subject_0[:, mask_b_0], axis=0)
    scan_data_normalized = scan_data_subject_0.astype("float32") * subject_0.lstsq_coefficient / subject_0.max_data

    store_dmri_data(scan_data_normalized, subject_0)

    for subject in subjects[1:]:
        logger.info("normalizing subject %s", subject.subject_name)
        scan_data = load_scan(subject)
        median_scan = np.median(scan_data[:, mask_b_0], axis=0)

        scan_lstsq_coef, _, _, _ = np.linalg.lstsq(median_scan[:, np.newaxis], scan_data_median_subject_0, rcond=-1)
        logger.info("lstsq coefficient for subject %s: %f", subject.subject_name, scan_lstsq_coef[0])

        subject.lstsq_coefficient = scan_lstsq_coef[0]
        subject.max_data = max_data

        scan_data_normalized = scan_data.astype("float32") * subject.lstsq_coefficient / subject.max_data

        store_dmri_data(scan_data_normalized, subject)

In [ ]:
normalize_data(mri_mudi_subjects)

In [ ]:
normalize_data(mri_hcp_subjects)

## Step 3: Average bvecs and create scheme file

In [ ]:
def average_bvecs(subjects: list[MRISubjectData]):
    bvecs = None
    other = None
    for subject in subjects:
        scheme = np.loadtxt(subject.scheme_path)
        if bvecs is not None:
            bvecs += scheme[:, :3]
        else:
            bvecs = scheme[:, :3]
            other = scheme[:, 3:]

    bvecs /= len(subjects)
    for i in range(bvecs.shape[0]):
        if other[i, 0] != 0.0:  # if b-value is not 0
            bvecs[i] /= np.sqrt(np.sum(bvecs[i] ** 2))
        else:
            bvecs[i] = 0.0

    return np.c_[bvecs, other]

In [ ]:
scheme = average_bvecs(mri_mudi_subjects)
np.savetxt(Path(MUDI_PATH, "scheme.txt"), scheme, fmt="%.6f %.6f %.6f %d %d %d")
with h5py.File(mri_mudi_subjects[0].output_file, "a") as hdf5_f:
    hdf5_f.create_dataset("scheme", data=scheme)

In [ ]:
scheme = average_bvecs(mri_hcp_subjects)
np.savetxt(Path(HCP_PATH, "scheme.txt"), scheme, fmt="%.6f %.6f %.6f %d")
with h5py.File(mri_hcp_subjects[0].output_file, "a") as hdf5_f:
    hdf5_f.create_dataset("scheme", data=scheme)

## Step 4: Generate White matter, grey matter, etc, masks

In [ ]:
def generate_5tt_mask(subject: MRISubjectData):
    """Docs: https://mrtrix.readthedocs.io/en/latest/reference/commands/5ttgen.html#ttgen-fsl"""
    logger.info("starting 5tt generation, this will take a few minutes...")
    output_path = Path(subject.root_path, "5tt.nii")
    result = subprocess.run(
        [
            "5ttgen",
            "fsl",
            Path(subject.root_path, subject.t1_path),  # The input T1-weighted image
            output_path,  # The output 5TT image
            "-mask",
            Path(subject.root_path, subject.brain_mask_path),
            "-force",
            "-nocrop",
        ]
    )
    if result.returncode != 0:
        logger.error("could not finish generation, return code: %d", result.returncode)
        return

    subject.fivett_mask_path = output_path
    logger.info("finished 5tt generation, return code: %d", result.returncode)
    return result.returncode

In [ ]:
def binarize_and_mask_5tt(subject: MRISubjectData):
    target_img = image.load_img(str(Path(subject.root_path, subject.brain_mask_path)))
    target_affine = target_img.affine
    target_shape = image.get_data(target_img).shape

    source_img = image.load_img(str(subject.fivett_mask_path))
    source_affine = source_img.affine
    source_shape = image.get_data(source_img).shape

    if not np.allclose(target_affine, source_affine):
        source_img = image.resample_img(source_img, target_affine=target_affine, target_shape=target_shape)

    source_img = image.binarize_img(source_img, threshold=1 / 5)

    return np.transpose(apply_mask(source_img, target_img))


def create_and_store_5tt(subjects):
    for subject in subjects:
        if subject.fivett_mask_path is None:
            returncode = generate_5tt_mask(subject)
            if returncode != 0:
                logger.error("could not 5tt masks")
                continue

        _5tt_data = binarize_and_mask_5tt(subject)

        with h5py.File(subject.output_file, "a") as file:
            if "5tt" not in file.keys():
                file.create_dataset("5tt", data=_5tt_data, chunks=True, maxshape=(None, _5tt_data.shape[1]))
            else:
                file["5tt"].resize(file["5tt"].shape[0] + _5tt_data.shape[0], axis=0)
                file["5tt"][-_5tt_data.shape[0] :] = _5tt_data

In [ ]:
create_and_store_5tt(mri_mudi_subjects)

## The MUDI data

In [ ]:
selected_imgs = image.index_img(str(Path(root_dir, "cdmri0011", img_file)), np.array([1, 10, 100, 1000]))

for img in image.iter_img(selected_imgs):
    # img is now an in-memory 3D img
    plotting.plot_anat(img)

Each scan has a mask to mask the brain out.

In [ ]:
plotting.plot_anat(str(Path(root_dir, "cdmri0011", msk_file)));

In [ ]:
scheme = np.loadtxt(Path(root_dir, "parameters_new.txt"))


def set_dir(r):
    if r[3] == 0.0:
        r[0:3] = 0.0
    return r


scheme = np.apply_along_axis(set_dir, 1, scheme)
fig = go.Figure(
    data=go.Scatter3d(
        x=scheme[:, 0],
        y=scheme[:, 1],
        z=scheme[:, 2],
        mode="markers",
        marker=dict(color=scheme[:, 3], colorscale="Bluered", showscale=True),
    )
)

fig.update_layout(
    title_text="Q-space colored by b-values.",
    margin=dict(l=0, r=0, b=0),
    width=1000,
    height=1000,
    scene=dict(
        annotations=[
            dict(
                showarrow=False,
                x=0,
                y=0,
                z=0,
                text="b = 0",
                xanchor="left",
                xshift=10,
                opacity=0.7,
            )
        ]
    ),
)
fig.show()

## Load data

In [ ]:
scans = list()  # cannot be numpy array as scans are irregular shaped.
for name in ["cdmri0011", "cdmri0012", "cdmri0013", "cdmri0014", "cdmri0015"]:
    scan_f_img = str(Path(root_dir, name, img_file))
    scan_f_msk = str(Path(root_dir, name, msk_file))
    scan = np.transpose(apply_mask(imgs=scan_f_img, mask_img=scan_f_msk))

    scans.append(scan)

    logger.debug("Loaded scan with shape: %s", scan.shape)

In [ ]:
fig, axes = plt.subplots(1, 5, sharey=True, figsize=(20, 10))
for i in range(5):
    axes[i].hist(scans[i].flatten())

### Distribution of the 95th percentile of the data

In [ ]:
max_data = np.percentile(scans[0].flatten(), 95)
logger.info("95th percentile max: %f", max_data)

In [ ]:
fig, axes = plt.subplots(1, 5, sharey=True, figsize=(20, 10))
for i in range(5):
    axes[i].hist(scans[i].flatten(), range=[0, max_data])

## Harmonize the data
We want the different scans to have a similair distribution of values. We look at different techniques to do this.

### Median across all images

In [ ]:
median_scans = list()
for i in range(5):
    median_scan = np.median(scans[i], axis=0)
    median_scans.append(median_scan)

In [ ]:
scan_lstsq_coefs = np.empty((5, 2), dtype=np.float32)
scan_lstsq_coefs[0] = np.asarray([1.0, 11])  # initialize with 1 because the coef of itself is 1

for i in range(1, 5):
    scan_lstsq_coef, _, _, _ = np.linalg.lstsq(median_scans[i][:, np.newaxis], median_scans[0], rcond=-1)
    scan_lstsq_coefs[i] = np.asarray([scan_lstsq_coef[0], 11 + i])

    logger.info("lstsq coefficient for scan %d: %f", 11 + i, scan_lstsq_coef)

In [ ]:
fig, axes = plt.subplots(1, 5, sharey=True, figsize=(20, 5))

colors = ["bo", "go", "ro", "co", "mo"]
for i in range(5):
    axes[i].plot(
        median_scans[i],
        median_scans[0],
        colors[i],
        median_scans[i],
        median_scans[i] * scan_lstsq_coefs[i, 0],
        ":k",
        median_scans[i],
        median_scans[i],
        "-k",
    )

for ax in axes:
    ax.set(aspect="equal")

### Median across b=0 images

In [ ]:
scheme = np.loadtxt(Path(root_dir, "parameters_new.txt"))
mask = scheme[:, 3] == 0.0

In [ ]:
median_scans = list()
for i in range(5):
    median_scan = np.median(scans[i][:, mask], axis=0)
    median_scans.append(median_scan)

In [ ]:
scan_lstsq_coefs = np.empty((5, 2), dtype=np.float32)
scan_lstsq_coefs[0] = np.asarray([1.0, 11])  # initialize with 1 because the coef of itself is 1

for i in range(1, 5):
    scan_lstsq_coef, _, _, _ = np.linalg.lstsq(median_scans[i][:, np.newaxis], median_scans[0], rcond=-1)
    scan_lstsq_coefs[i] = np.asarray([scan_lstsq_coef[0], 11 + i])

    logger.info("lstsq coefficient for scan %d: %f", 11 + i, scan_lstsq_coef)

In [ ]:
fig, axes = plt.subplots(1, 5, sharey=True, figsize=(20, 5))

colors = ["bo", "go", "ro", "co", "mo"]
for i in range(5):
    axes[i].plot(
        median_scans[i],
        median_scans[0],
        colors[i],
        median_scans[i],
        median_scans[i] * scan_lstsq_coefs[i, 0],
        ":k",
        median_scans[i],
        median_scans[i],
        "-k",
    )

for ax in axes:
    ax.set(aspect="equal")

This technique seems to get us closer to scan 0, so we use this.

### Normalize the data

#### Normalize according to 99 percentile of Subject 11 and save in one big file

In [ ]:
# max_data = masked_data11.max()
max_data = np.percentile(masked_data11, 99)
masked_data11n = masked_data11.astype("float32") / max_data
masked_data11n = np.clip(masked_data11n, 0, 1)
masked_data12n = masked_data12.astype("float32") * a12 / max_data
masked_data12n = np.clip(masked_data12n, 0, 1)
masked_data13n = masked_data13.astype("float32") * a13 / max_data
masked_data13n = np.clip(masked_data13n, 0, 1)
masked_data14n = masked_data14.astype("float32") * a14 / max_data
masked_data14n = np.clip(masked_data14n, 0, 1)
masked_data15n = masked_data15.astype("float32") * a15 / max_data
masked_data15n = np.clip(masked_data15n, 0, 1)

In [ ]:
print(max_data)
print(masked_data11.shape)

In [ ]:
# plt.hist(masked_data11, bins = 'auto')
# plt.show()

In [ ]:
subj11 = 11 * np.ones((masked_data11.shape[0],), dtype=int)
subj12 = 12 * np.ones((masked_data12.shape[0],), dtype=int)
subj13 = 13 * np.ones((masked_data13.shape[0],), dtype=int)
subj14 = 14 * np.ones((masked_data14.shape[0],), dtype=int)
subj15 = 15 * np.ones((masked_data15.shape[0],), dtype=int)

In [ ]:
subj = np.concatenate((subj11, subj12, subj13, subj14, subj15), axis=0)
print(subj.shape)
masked_data = np.concatenate(
    (masked_data11n, masked_data12n, masked_data13n, masked_data14n, masked_data15n),
    axis=0,
)
print(masked_data.shape)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(np.concatenate((subj[:, np.newaxis], masked_data), axis=1))

In [ ]:
df

In [ ]:
df.to_csv("data.csv")

#### Normalize according to 95 percentile of Subject 11 and save as separate files

In [ ]:
# max_data = masked_data11.max()
max_data = np.percentile(masked_data11, 95)
masked_data11n = masked_data11.astype("float32") / max_data
masked_data11n = np.clip(masked_data11n, 0, 1)
masked_data12n = masked_data12.astype("float32") * a12 / max_data
masked_data12n = np.clip(masked_data12n, 0, 1)
masked_data13n = masked_data13.astype("float32") * a13 / max_data
masked_data13n = np.clip(masked_data13n, 0, 1)
masked_data14n = masked_data14.astype("float32") * a14 / max_data
masked_data14n = np.clip(masked_data14n, 0, 1)
masked_data15n = masked_data15.astype("float32") * a15 / max_data
masked_data15n = np.clip(masked_data15n, 0, 1)

In [ ]:
print(max_data)
print(masked_data11.shape)

In [ ]:
fig, axes = plt.subplots(1, 5, sharey=True, figsize=(20, 10))
axes[0].hist(masked_data11n.flatten(), range=[0, 1])
axes[1].hist(masked_data12n.flatten(), range=[0, 1])
axes[2].hist(masked_data13n.flatten(), range=[0, 1])
axes[3].hist(masked_data14n.flatten(), range=[0, 1])
axes[4].hist(masked_data15n.flatten(), range=[0, 1])
fig.show()

In [ ]:
import pandas as pd

In [ ]:
df11 = pd.DataFrame(np.concatenate((np.zeros_like(subj11[:, np.newaxis]), masked_data11n), axis=1))
df12 = pd.DataFrame(np.concatenate((np.zeros_like(subj12[:, np.newaxis]), masked_data12n), axis=1))
df13 = pd.DataFrame(np.concatenate((np.zeros_like(subj13[:, np.newaxis]), masked_data13n), axis=1))
df14 = pd.DataFrame(np.concatenate((np.zeros_like(subj14[:, np.newaxis]), masked_data14n), axis=1))
df15 = pd.DataFrame(np.concatenate((np.zeros_like(subj15[:, np.newaxis]), masked_data15n), axis=1))

In [ ]:
df11

In [ ]:
df11.to_csv("data11.csv")
df12.to_csv("data12.csv")
df13.to_csv("data13.csv")
df14.to_csv("data14.csv")
df15.to_csv("data15.csv")

#### Normalize according to 95 percentile of Subject 11, don't clip, and save in two big files (one 'header' and one 'data') (USE THIS)

In [ ]:
max_data = np.percentile(scans[0], 95)
logger.info("95th percentile: %f", max_data)

normalized_scans = list()
for i in range(5):
    normalized_scan = scans[i].astype("float32") * scan_lstsq_coefs[i, 0] / max_data
    normalized_scans.append(normalized_scan)

In [ ]:
fig, axes = plt.subplots(1, 5, sharey=True, figsize=(20, 10))

for i in range(5):
    axes[i].hist(normalized_scans[i].flatten(), range=[0, 1])

Save the scan data in one file. This makes it easier later for training. We save a seperate header file such that we can distinguish the different scans from each other later. 

In [ ]:
def save_data(name: str, max_rows: Optional[int] = None):
    """Save the data to hdf5

    Args:
        name (str): name of the file.
        max_rows (int, optional): use this to select a subset of the data. Used
        for testing. Set to None to get all the data. Defaults to None.
    """
    indexes = np.concatenate(
        [(i + 11) * np.ones((scans[i].shape[0],), dtype=int)[:max_rows] for i in range(5)],
        axis=0,
    )
    data = np.concatenate(
        [normalized_scan[:max_rows, :] for normalized_scan in normalized_scans],
        axis=0,
    )

    hdf5_f_path = Path(root_dir, f"{name}.hdf5")
    with h5py.File(hdf5_f_path, "w") as hdf5_f:
        hdf5_f.create_dataset("data", data=data)
        hdf5_f.create_dataset("index", data=indexes)
        hdf5_f.create_dataset("scheme", data=scheme)

        normalization_data = hdf5_f.create_group("normalization_data")
        normalization_data.create_dataset("lstsq_coef", data=scan_lstsq_coefs)
        normalization_data.create_dataset("max_data", data=np.asarray([max_data]))

Save some fake data to test the correctness of the neural network

In [ ]:
save_data("data_fake", max_rows=500)

Proceed saving the real data

In [ ]:
save_data("data")

# Test data

In [ ]:
selecf = "/home/sapct5/Documents/Code/MUDI/MUDI_CA_LR/Run(lr=0.001, batch_size=100)K=500_epoch=800_testnone_unique.txt"
selecind = np.sort(np.loadtxt(selecf, dtype=int))
print(selecind)

mask = scheme[:, 3] == 0.0

mask_ = np.zeros(np.shape(mask), dtype=bool)
mask_[selecind] = True
print(mask_.shape)

mask3 = mask & mask_
print(mask3.shape)

scheme_ = scheme[mask_]
print(scheme_.shape)

mask2 = scheme_[:, 3] == 0.0
print(mask2.shape)

In [ ]:
direc16 = "./data"
masked_data16 = np.transpose(
    apply_mask(
        imgs=os.path.join(direc16, "16_MB_RE_t.nii.gz"),
        mask_img=os.path.join(direc16, "brain_mask-testing1.nii.gz"),
    )
)

In [ ]:
masked_data16.shape

In [ ]:
direc17 = "./data"
masked_data17 = np.transpose(
    apply_mask(
        imgs=os.path.join(direc17, "17_MB_RE_t.nii.gz"),
        mask_img=os.path.join(direc17, "brain_mask-testing2.nii.gz"),
    )
)

In [ ]:
med16 = np.median(masked_data16[:, mask2], axis=0)
med17 = np.median(masked_data17[:, mask2], axis=0)

In [ ]:
med11_ = np.median(masked_data11[:, mask3], axis=0)

In [ ]:
a16, _, _, _ = np.linalg.lstsq(med16[:, np.newaxis], med11_)
a17, _, _, _ = np.linalg.lstsq(med17[:, np.newaxis], med11_)
print(a16, a17)

In [ ]:
fig, axes = plt.subplots(1, 2, sharey=True, figsize=(20, 5))
axes[0].plot(med16, med11_, "yo", med16, med16 * a16, ":k", med16, med16, "-k")
axes[1].plot(med17, med11_, "bo", med17, med17 * a17, ":k", med17, med17, "-k")
for ax in axes:
    ax.set(aspect="equal")
fig.show()

In [ ]:
selected_imgs = image.index_img(os.path.join(direc11, img_file), np.array(selecind[[1, 10, 100, 300]]))
for img in image.iter_img(selected_imgs):
    # img is now an in-memory 3D img
    plotting.plot_anat(img, vmin=0, vmax=15)

In [ ]:
selected_imgs = image.index_img(os.path.join(direc16, "16_MB_RE_t.nii.gz"), np.array([1, 10, 100, 300]))
for img in image.iter_img(selected_imgs):
    # img is now an in-memory 3D img
    plotting.plot_anat(img, vmin=0, vmax=15)

In [ ]:
masked_data16n = masked_data16.astype("float32") * a16 / max_data
masked_data17n = masked_data17.astype("float32") * a17 / max_data

In [ ]:
subj16 = 16 * np.ones((masked_data16.shape[0],), dtype=int)
subj17 = 17 * np.ones((masked_data17.shape[0],), dtype=int)

In [ ]:
subj = np.concatenate((subj16, subj17), axis=0)
print(subj.shape)
masked_data = np.concatenate((masked_data16n, masked_data17n), axis=0)
print(masked_data.shape)

In [ ]:
ind = np.arange(len(subj16) + len(subj17))
ind.shape

In [ ]:
df1 = pd.DataFrame(np.concatenate((ind[:, np.newaxis], subj[:, np.newaxis]), axis=1))

In [ ]:
df1.to_csv("header_test.csv")

In [ ]:
import h5py

h5f = h5py.File("data_test.hdf5", "w")
h5f.create_dataset("data1", data=masked_data)
h5f.close

# (not working yet) Log transform, normalise according to 95 percentile of Subject 11, don't clip, and save in two big files (one 'header' and one 'data')

In [ ]:
# max_data = masked_data11.max()
max_data = np.percentile(masked_data11, 95)
masked_data11n = masked_data11.astype("float32") / max_data
masked_data12n = masked_data12.astype("float32") * a12 / max_data
masked_data13n = masked_data13.astype("float32") * a13 / max_data
masked_data14n = masked_data14.astype("float32") * a14 / max_data
masked_data15n = masked_data15.astype("float32") * a15 / max_data

In [ ]:
print(max_data)
print(masked_data11.shape)

In [ ]:
fig, axes = plt.subplots(1, 5, sharey=True, figsize=(20, 10))
axes[0].hist(masked_data11n.flatten(), range=[0, 1])
axes[1].hist(masked_data12n.flatten(), range=[0, 1])
axes[2].hist(masked_data13n.flatten(), range=[0, 1])
axes[3].hist(masked_data14n.flatten(), range=[0, 1])
axes[4].hist(masked_data15n.flatten(), range=[0, 1])
fig.show()

In [ ]:
subj11 = 11 * np.ones((masked_data11.shape[0],), dtype=int)
subj12 = 12 * np.ones((masked_data12.shape[0],), dtype=int)
subj13 = 13 * np.ones((masked_data13.shape[0],), dtype=int)
subj14 = 14 * np.ones((masked_data14.shape[0],), dtype=int)
subj15 = 15 * np.ones((masked_data15.shape[0],), dtype=int)

In [ ]:
subj = np.concatenate((subj11, subj12, subj13, subj14, subj15), axis=0)
print(subj.shape)
masked_data = np.concatenate(
    (masked_data11n, masked_data12n, masked_data13n, masked_data14n, masked_data15n),
    axis=0,
)
print(masked_data.shape)

In [ ]:
ind = np.arange(len(subj11) + len(subj12) + len(subj13) + len(subj14) + len(subj15))
ind.shape

In [ ]:
import pandas as pd

In [ ]:
df1 = pd.DataFrame(np.concatenate((ind[:, np.newaxis], subj[:, np.newaxis]), axis=1))

In [ ]:
df1.to_csv("header_.csv")

In [ ]:
df1

In [ ]:
df = pd.DataFrame(masked_data)

In [ ]:
df.to_csv("data_.csv")